In [ ]:
import httpx
import os
from dotenv import load_dotenv

load_dotenv()

def find_problem(problem_id: int):
    api_url = f'{os.getenv("DOCKER_API_BASE_URL")}/problems/{problem_id}' if os.path.exists('/.dockerenv') else f'{os.getenv("API_BASE_URL")}/problems/{problem_id}'
    headers = { 'Content-Type': 'application/json' }
    try:
        response = httpx.get(api_url, headers=headers)
        if 200 <= response.status_code < 300:
            print(f'✅ Successfully found problem id {problem_id}')
            return response.json()
        else:
            print(f'❌ Failed to get problem id {problem_id} with status code {response.status_code}')

    except httpx.RequestError as e:
        print(f'🚨 Error sending request: {e}')

problem_data = find_problem(1)

user_submission = '''
    Use recursion to count all possible ways
    At each step, either take 1 step or 2 steps
    The base case is when n is 0 or 1
    Then, return the sum of the ways for n-1 and n-2
'''

prompt = f'''
    You are a **professor specializing in coding problems**. The user will describe their approach to solving a problem, and you will **analyze their response and provide structured feedback**.

    ### Instructions:
    - Provide a **concise analysis** (one paragraph).
    - List **improvement suggestions** (bullet format).
    - Assign a **score** as an integer:
    - **1** → Incorrect approach.
    - **2** → Partially correct approach.
    - **3** → Correct approach.
    - **Talk directly to the user using "you" instead of referring to them in the third person.**
    - **Follow the exact format in the "Expected Output" section. Do not change the order.**

    ### Problem Details:
    **Title:** {problem_data['title']}
    **Difficulty:** {problem_data['difficulty']}
    **Description:** {problem_data['description']}
    **Constraints:** {''.join([f'{constraint}' for constraint in problem_data['constraints']])}
    **Examples:** {chr(10).join([f'Input: {example['input']}, Output: {example['output']}, Explanation: {example['explanation']}' for example in problem_data['examples']])}
    
    ### User Submission: {user_submission}

    ### **Expected Output (Follow This Exactly)**
    **Analysis:**  
    <One-paragraph analysis of the user's approach>  

    **Suggestions:**  
    - <Bullet point suggestions>  

    **Score:** <Integer (1, 2, or 3)>
'''

# print(prompt)


✅ Successfully found problem id 1


In [ ]:
from ollama import Client

# Must have ollama on in the background
def ask_model(prompt: str):
    try:
        response = Client().generate(model='deepseek-r1:8b', prompt=prompt)
        print(response['response'])
    except Exception as e:
        print(f'🚨 Error: {e}')

ask_model(prompt)


<think>
Okay, let's break down the user's approach to solving the Climbing Stairs problem. The user suggests using recursion where at each step you can take either one or two steps. They mention that the base cases are when n is 0 or 1, and then return the sum of the ways for n-1 and n-2.

Hmm, I remember that this problem is a classic example of dynamic programming because it has overlapping subproblems. The recursive approach they mentioned would work in theory but might not be efficient for larger values of n due to repeated calculations. For instance, if n is 45, the recursion could lead to a lot of redundant calls, making the solution very slow.

Wait, the user's idea is correct in terms of the logic because each step can indeed be broken down into smaller subproblems where you add the ways from n-1 and n-2. That aligns with the dynamic programming approach. So their approach is conceptually right but might have performance issues without memoization or an iterative method.

I sho